In [1]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
import ollama
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions
import time
import multiprocessing as mp
from sentence_transformers import SentenceTransformer, util

from langchain.prompts import PromptTemplate
from operator import itemgetter

h:\Other computers\My Computer\Work\LLM\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docdir='H:/Other computers/My Computer/Work/LLM/rag_docs/'
dbdir='C:/Users/Renhwai/Documents/Local Projects/vectorstore/'
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [3]:
for i in os.listdir(docdir):
    print(i.replace('.txt',''))

2023 Feb Data Analyst Resume.docx.pdf
which are the technical tools that you have used for analysis and presentation purposes
what are the best methods for data cleaning
explain descriptive, predictive, and prescriptive analytics
what are some common data visualization tools you have used
how can you handle missing values in a dataset
what is Time Series analysis
what is Overfitting
why should we hire you
please share about your data analysis projects from your past working experience
please share about your development projects
what are your strengths and weaknesses as a data analyst


In [ ]:
docsplit=TextLoader(docdir+'Job descriptions of opening position shared by Hiring Manager.txt').load_and_split(text_splitter)

In [ ]:
docsplit

In [6]:
def create_client_collection(dbdir,collectioname):
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
    chroma_client=chromadb.PersistentClient(path=dbdir+collectioname)
    return chroma_client,chroma_client.create_collection(name=collectioname, embedding_function=sentence_transformer_ef)

In [7]:
client1,collection1=create_client_collection(dbdir,'collection1')

In [66]:
pdf=PyPDFLoader(docdir+'2023 Feb Data Analyst Resume.docx.pdf').load()

In [69]:
chunk = text_splitter.split_documents(pdf)

[Document(page_content="KONG REN HW AI (Mr .)\nSUMMARY\nA skilled and experienced process engineer with 7 years in the \nSemiconductor Industry. Achieved a Bachelor's Degree in Materials \nEngineering with First Class Honours and completed Google Data Analytics \ncertiﬁcation\n(\nlink\n)\n.\nProﬁcient in big data analytics, agile project management, and data analysis \ntechniques. Seeking new opportunities in Big Data Analytics and AI \ninnovation.\nWORK EXPERIENCE\nMicron Singapore Backend, Singapore", metadata={'source': 'H:/Other computers/My Computer/Work/LLM/rag_docs/2023 Feb Data Analyst Resume.docx.pdf', 'page': 0}), Document(page_content='innovation.\nWORK EXPERIENCE\nMicron Singapore Backend, Singapore\nSenior Industry 4.0 Analyst  (March 2022 -\nPresent)\n●\nManage stakeholder expectations and user requirements from VP, director and engineer\n●\nLed projects in Tool Alarm KPI Management, Virtual Metrology, Fault Detection Control & \nPredictive Maintenance\n●\nTrained in Jira

In [ ]:
for i in chunk:
    print(i.page_content)

In [8]:
def update_docfolder2(db_collection,folderpath):
        print(db_collection,'is generating...')
        id=[]
        index=0
        for i in os.listdir(folderpath):
            if i.endswith(".txt") and i!='Job descriptions of opening position shared by Hiring Manager.txt':
                txt=TextLoader(folderpath+i,encoding='utf8').load_and_split(text_splitter)
                #txt=TextLoader(folderpath+i).load()
                topic=i.replace(".txt","")
                for chunk in txt:
                    id=[str(index)]
                    db_collection.add(
                    documents=chunk.page_content,
                    metadatas=[{"topic":topic,'source':chunk.metadata['source']}],
                    ids=id
                    )
                    index=index+1
                print(i,"is added.")
                
            if i.endswith(".txt") and i=='Job descriptions of opening position shared by Hiring Manager.txt':
                txt=TextLoader(folderpath+i,encoding='utf8').load_and_split(text_splitter)
                #txt=TextLoader(folderpath+i).load()
                topic=i.replace(".txt","")
                for chunk in txt:
                    id=['-'+str(index)]
                    db_collection.add(
                    documents=chunk.page_content,
                    metadatas=[{"topic":topic,'source':chunk.metadata['source']}],
                    ids=id
                    )
                    index=index+1
                print(i,"is added.")
                
            if i.endswith(".pdf"):
                pdf=PyPDFLoader(folderpath+i).load()
                chunk = text_splitter.split_documents(pdf)
                topic=i.replace(".pdf","")
                for j in chunk:
                    id=[str(index)]
                    db_collection.add(
                    documents=j.page_content,
                    metadatas=[{"topic":topic,'source':j.metadata['source']}],
                    ids=id
                    )
                    index=index+1
                print(i,"is added.")
            

In [9]:
update_docfolder2(collection1,docdir)

name='collection1' id=UUID('d48ef7d8-5655-406b-8ecc-80299ea8f19e') metadata=None tenant='default_tenant' database='default_database' is generating...
2023 Feb Data Analyst Resume.docx.pdf is added.
which are the technical tools that you have used for analysis and presentation purposes.txt is added.
what are the best methods for data cleaning.txt is added.
explain descriptive, predictive, and prescriptive analytics.txt is added.
what are some common data visualization tools you have used.txt is added.
how can you handle missing values in a dataset.txt is added.
what is Time Series analysis.txt is added.
what is Overfitting.txt is added.
why should we hire you.txt is added.
please share about your data analysis projects from your past working experience.txt is added.
please share about your development projects.txt is added.
what are your strengths and weaknesses as a data analyst.txt is added.


In [157]:
def update_docfolder(db_collection,folderpath):
        print(db_collection,'is generating...')
        id=[]
        index=0
        for i in os.listdir(folderpath):
            if i.endswith(".txt") and i!='Job descriptions of opening position shared by Hiring Manager.txt':
                txt=TextLoader(folderpath+i).load()
                q=i.replace(".txt","")
                id=[str(index)]
                db_collection.add(
                documents=txt[0].page_content,
                metadatas=[{"topic":q,'source':txt[0].metadata['source']}],
                ids=id
                )
                print(q,"is added.")
                index=index+1
            if i.endswith(".txt") and i=='Job descriptions of opening position shared by Hiring Manager.txt':
                txt=TextLoader(folderpath+i).load()
                #id=[str(index)]
                q=i.replace(".txt","")
                db_collection.add(
                documents=txt[0].page_content,
                metadatas=[{"topic":q,'source':txt[0].metadata['source']}],
                ids="-1"
                )
                print(q,"is added.")
                #index=index+1
            if i.endswith(".pdf"):
                pdf=PyPDFLoader(docdir+i).load()
                chunk = text_splitter.split_documents(pdf)
                for j in chunk:
                    id=[str(index)]
                    q=i.replace(".pdf","")
                    db_collection.add(
                    documents=chunk[0].page_content,
                    metadatas=[{"topic":q,'source':chunk[0].metadata['source']}],
                    ids=id
                    )
                    print(q,"is added.")
                    index=index+1
            

In [ ]:
update_docfolder(collection,docdir)

In [59]:
dbclient=chromadb.PersistentClient(path="H:/Other computers/My Computer/Work/LLM/vectorstore/collection8")
#sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
#db10=dbclient.get_collection(name='collection4', embedding_function=sentence_transformer_ef)
vectorstore = Chroma(
    client=dbclient,
    collection_name="collection8",
    embedding_function=SentenceTransformerEmbeddings(),
    )

In [64]:
print(vectorstore.similarity_search('What did the manager ask?')[0].page_content)

Hiring Manager asks, please share about your data analysis projects from your past working experience. The answer is, during my time as a Senior Industrial 4.0 Analyst at Micron Semiconductor, I successfully managed smart manufacturing projects focused on smart manufacturing and digital transformation. Here are a few examples:


In [ ]:
txt=TextLoader('H:/Other computers/My Computer/Work/LLM/rag_docs/Job descriptions of opening position shared by Hiring Manager.txt').load()
vectorstore.update_document(
    document_id="-1",
    document=txt[0]
)

In [163]:
vectorstore.similarity_search('Tell me about this job description')

[Document(page_content="Hiring Manager from Klook seeks a Data Analyst with at least 4 years of experience in data analysis, data modeling, and understanding intricate business processes. The ideal candidate possesses strong technical skills in SQL, data querying languages, and data visualization tools. They must have excellent communication skills and experience collaborating with cross-functional teams. The role involves analyzing business requirements, documenting and scrutinizing business processes, and creating data models. The successful candidate will play a crucial role in driving Klook's data-driven initiatives and ensuring the success of data-driven products and services.", metadata={'source': 'H:/Other computers/My Computer/Work/LLM/rag_docs/Job descriptions of opening position shared by Hiring Manager.txt', 'topic': 'Job descriptions of opening position shared by Hiring Manager'}),
 Document(page_content="Hiring Manager asks question, what are your strengths and weaknesses 

In [70]:
job_description="""Senior Data Analyst
About Agoda

Agoda is an online travel booking platform for accommodations, flights, and more. We build and deploy cutting-edge technology that connects travelers with more than 3.6 million accommodations globally. Based in Asia and part of Booking Holdings, our 6,000+ employees representing 90+ nationalities foster a work environment rich in diversity, creativity, and collaboration. We innovate through a culture of experimentation and ownership, enhancing the ability for our customers to experience the world.

 Our Purpose –  Bridging the World Through Travel 

We believe travel allows people to enjoy, learn and experience more of the amazing world we live in. It brings individuals and cultures closer together, fostering empathy, understanding and happiness.

We are a skillful, driven and diverse team from across the globe, united by a passion to make an impact. Harnessing our innovative technologies and strong partnerships, we aim to make travel easy and rewarding for everyone.

 Get to Know our  Team: 

The Performance Marketing Team of Agoda is a world leader in online marketing. This department is highly data-driven and focused on developing at-scale marketing programs that improve the lifetime value of Agoda customers through measurable marketing programs and channels. The team is a blend of the best analysts, marketing strategists, and data scientists in the world. The marketing leadership at Agoda have deep experience in data science, product, strategy, and other marketing fields and have built an organization that thrives on data, creative ideas, and technology. The Performance Marketing Team also fosters a great learning environment. You will be able to learn and grow by working closely with experts from a variety of backgrounds from all over the world.

 In this Role, you’ll get  to: 

Search: Experiment with text ads, bidding, and campaign structures on Google, Bing, Baidu, Naver, and other search engines. Adapt to new product features and roll out changes from successful tests
Display: Test, analyze, and optimize campaigns on Facebook, Twitter, Instagram, and others
Modeling: Analyze the vast amounts of data generated by experiments, develop models we can use for optimization, and build dashboards for account managers

 What you’ll Need to  Succeed: 

Bachelor’s Degree or higher from top university in a quantitative subject (computer science, mathematics, engineering, statistics or science)
Ability to communicate fluently in English
Good numerical reasoning skills
Proficiency in Excel
Intellectual curiosity

 It’s Great if you  Have: 

Exposure to one or more data analysis packages or databases, e.g., SAS, R, SPSS, Python, VBA, SQL
Experience in digital marketing
Academic research experience
        
    """

def job_gap(job_description):
    llm = Ollama(
        model='mistral:7b-instruct',
        #model='llama2:chat',
        #model='phi',
        #model='falcon:instruct',
        num_predict=4096,
        temperature=0.8,
        top_k=40,
        )
    
    candidate_resume="""
    Agile Project Management: Experience with agile & waterfall hybrid project management using Jira and Confluence
    Python: Familiar with Python data analysis and modeling using Scikit, Matlib, Plotty. Also have experience in using LLM framework Ollama, Langchain and Streamlit
    Big Data and Data Visualization : Proficient in basic SQL, Snowflake, Bigquery, Spotfire and Tableau
    Data Analysis: Familiar with data cleaning, hypothesis test (eg: Normality, ANOVA) and visualization using Python, JMP and Excel. Passionate in diving data set and exploration to extract data insights. Complete Google Data Analytics certification
    Manufacturing Process Expert: Familiar in source of variation and factor correlation study for root cause finding and Measurement System Analysis (MSA)
    Design of Experiment (DOE): Completed JMP Classical and Custom DOE courses. Certified as Company DOE Trainer
    Kepner Tregoe (KT) Problem Solving: Complete KT training with a certified trainer, utilizing logical thinking and data analysis to analyze root cause and perform risk assessment
    Microsoft Sharepoint: Owner of department collaboration site. Responsible for KPI reporting, project tracking, document permission & retention control
    Microsoft Office: Expert in Microsoft Offices, include PowerPoint and Excel
    Statiscal Process Control (SPC): Process SPC owner and FMEA champion
    8D Problem Solving and Reporting: Experienced with 8D report requirements for internal and external customers
    Semiconductor: More than 7 years exprience in Micron Semiconductor. Expert in Wire Bond(WB) process and Package Assembly. Held WB Process Engineer positions from HVM, NPI and TD. Collaborate with both internal product and business unit (BU) team and external key equipment group (KEG), OSAT and supplier (eg: wire, substrate) for new prodct development and reliability qualification
    Stakeholder Communication and Management: Experience in collaboration with executives and crosss-functional teams. Provide project updata or data presentation to directos and vice presidents
    Material Engineering: Bachelor of Materials Engineering with First Class Honours from Nanyang Technological University, Singapore
    Finance: I have complete CFA Level 1 and currently I am candidate for Level 2
    """

    prompt_template=""""
        You are job recruiter. You are performing job skill gap analysis of a job candidate step by step:
        Step 1: Based on {job_description}, list down your job posiion name, company name, job responsibilities and job requirements,
        Step 2: Based on {candidate_resume}, list down the candidate skills,
        Step 3: Finaly, compare Step 1 and Step 2, and identify any skill gaps
        """
    prompt_template = PromptTemplate.from_template(prompt_template)

    chain =(prompt_template
            |llm
            |StrOutputParser()
            )
        
    stream=chain.invoke(
        {"job_description":job_description,
         "candidate_resume":candidate_resume,}
         )

    text_file=open(docdir+'Job descriptions of opening position shared by Hiring Manager.txt','w')
    text_file.write(stream)
    text_file.close()
    print(stream)

In [71]:
job_gap(job_description)

 Based on the information provided in both steps, here is a comparison of the job position and requirements between being a Senior Data Analyst at Agoda and having various other skill sets:

**Job Position:** Senior Data Analyst at Agoda

**Company Name:** Agoda

**Job Responsibilities:**
- Search: Experiment with text ads, bidding, and campaign structures on search engines. Adapt to new product features and roll out changes from successful tests.
- Display: Test, analyze, and optimize campaigns on social media platforms.
- Modeling: Analyze data generated by experiments, develop models for optimization, and build dashboards for account managers.

**Job Requirements:**
- Bachelor's Degree or higher from a top university in a quantitative subject.
- Fluent English communication skills.
- Good numerical reasoning skills.
- Proficiency in Excel.
- Intellectual curiosity.

**Candidate Skills (Step 2):**
- Agile & waterfall hybrid project management using Jira and Confluence.
- Python data 